In [43]:
import json
import re
import numpy as np
import scipy as sp
import sklearn
import pickle

In [44]:
matchCount = 0
matchList = []
pattern = re.compile(r"(\d{8,10})\s+({)")
with open("../../data/pro_matches.txt") as f:
    buf = ""
    for oneline in f:
        m = pattern.match(oneline)
        if m:
            matchCount += 1
            if buf:
                matchList.append(json.loads(buf))
            buf = m.group(2)
                
        else:
            buf += oneline
if buf:
    matchList.append(json.loads(buf))
print matchCount

20999


In [45]:
# get heros win-rate
hero_wr = {}
with open("../../data/heros.txt") as f:
    for line in f:
        (hero_id,_,_,wr) = line.split()
        hero_id = int(hero_id)
        wr = float(wr.strip().rstrip("%"))/100.0
        hero_wr.update({hero_id:wr})
        
# get match up comb
comb = np.zeros([120,120])
anti = np.zeros([120,120])
with open("../../data/match_up_comb.txt") as f:
    for line in f:
        a, b, c = line.split()
        a = int(a)
        b = int(b)
        c = float(c)
        comb[a,b] = c
# get match up anti
with open("../../data/match_up_anti.txt") as f:
    for line in f:
        a, b, c = line.split()
        a = int(a)
        b = int(b)
        c = float(c)
        anti[a,b] = c

In [83]:
def ca(a, b):
    res = []
    for i in a:
        for j in a:
            if not i == j:
                res.append(comb[i,j])
        for j in b:
            res.append(anti[i,j])
    res = np.array(res)
    return res

def json2feat(a):
    teams = np.zeros([2,120], dtype=np.float)
    wr = np.zeros([2,5], dtype=np.float)
    playerCount = 0
    hl = np.zeros([2,5], dtype=int)
    cc = set()
    try:
        for player in a['result']['players']:
            teams[playerCount / 5][player['hero_id']] = 1
            #hero win rate
            wr[playerCount/5][playerCount%5] = hero_wr[player['hero_id']]
            hl[playerCount/5][playerCount%5] = player['hero_id']
            cc.add(player['hero_id'])
            playerCount += 1
        if not len(cc) == 10:
            return (False, False, False)
    except:
        return (False, False, False)
            
    p = np.zeros([2,45])
    p[0] = ca(hl[0], hl[1])
    p[1] = ca(hl[1], hl[0])
    ret = np.hstack([teams, wr, p]).reshape([1,-1])
    return (True, ret, np.array(a['result']['radiant_win']))

In [85]:
X = []
y = []
for match in matchList:
    if "error" in match['result'].keys():
        continue
    st, feat, res = json2feat(match)
    if not st:
        continue
    X.append(feat)
    y.append(res)
X = np.vstack(X)
y = np.vstack(y).flatten()
print X.shape
print y.shape

(20626, 340)
(20626,)


In [86]:
feat = (X,y)
pickle.dump(feat, open("../../data/feat","w"))